In [1]:
import pandas as pd
import numpy as np
import calendar
import warnings
from numba import jit 
import datetime
from time import time
from tqdm import tqdm_notebook as tqdm
from collections import Counter
from sklearn import preprocessing
from sklearn.model_selection import GroupKFold, StratifiedKFold
from sklearn.metrics import confusion_matrix, cohen_kappa_score
from typing import Any
import lightgbm as lgb
import json
from sklearn import metrics
from itertools import product
import copy
import time
from functools import partial
import scipy as sp
from scipy import stats
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows",1000)
np.set_printoptions(precision=8)
warnings.filterwarnings("ignore")

In [2]:
def quadratic_weighted_kappa(y_pred, data):
    y_pred = np.transpose(y_pred.reshape(4,-1))
    y_pred = np.argmax(y_pred, axis=1)
    y_true = np.array(data.get_label()).astype("int")
    c_matrix = confusion_matrix(y_true, y_pred, labels=[0,1,2,3])

    numer = 0.0
    denom = 0.0
    
    for i in range(c_matrix.shape[0]):
        for j in range(c_matrix.shape[1]):
            n = c_matrix.shape[0]
            wij = ((i-j)**2.0)
            oij = c_matrix[i, j]
            eij = c_matrix[i, :].sum() * c_matrix[:, j].sum() / c_matrix.sum()
            numer += wij * oij
            denom += wij * eij
    return "Quadratic weighted Kappa", 1 - numer / denom, True

In [3]:
def qwk(a1, a2):
    max_rat = 3
    a1 = np.asarray(a1, dtype=int)
    a2 = np.asarray(a2, dtype=int)

    hist1 = np.zeros((max_rat + 1, ))
    hist2 = np.zeros((max_rat + 1, ))

    o = 0
    for k in range(a1.shape[0]):
        i, j = a1[k], a2[k]
        hist1[i] += 1
        hist2[j] += 1
        o +=  (i - j) * (i - j)

    e = 0
    for i in range(max_rat + 1):
        for j in range(max_rat + 1):
            e += hist1[i] * hist2[j] * (i - j) * (i - j)

    e = e / a1.shape[0]

    return np.round(1 - o / e, 8)

In [4]:
class OptimizedRounder(object):
    def __init__(self):
        self.coef_ = 0

    def _kappa_loss(self, coef, X, y):
        X_p = pd.cut(X, [-np.inf] + list(np.sort(coef)) + [np.inf], labels = [0, 1, 2, 3])
        return -qwk(y, X_p)
    
    def fit(self, X, y):
        loss_partial = partial(self._kappa_loss, X=X, y=y)
        initial_coef = [0.5, 1.5, 2.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method='nelder-mead')
        
    def predict(self, X, coef):
        return pd.cut(X, [-np.inf] + list(np.sort(coef)) + [np.inf], labels = [0, 1, 2, 3])

    def coefficients(self):
        return self.coef_['x']

# install

In [5]:
%%time
train = pd.read_csv('../input/data-science-bowl-2019/train.csv')
train_labels = pd.read_csv('../input/data-science-bowl-2019/train_labels.csv')
test = pd.read_csv('../input/data-science-bowl-2019/test.csv')
#specs = pd.read_csv('../input/data-science-bowl-2019/specs.csv')
sample_submission = pd.read_csv('../input/data-science-bowl-2019/sample_submission.csv')

CPU times: user 1min 12s, sys: 8.31 s, total: 1min 21s
Wall time: 1min 21s


In [6]:
keep_id = train[train.type == "Assessment"][['installation_id']].drop_duplicates()
train = pd.merge(train, keep_id, on="installation_id", how="inner")
train = train[train.installation_id.isin(train_labels.installation_id.unique())]

# Preprocess and Feature engineering

In [7]:
%%time
def encode_title(train, test, train_labels):
    train['title_event_code'] = list(map(lambda x, y: str(x) + '_' + str(y), train['title'], train['event_code']))
    test['title_event_code'] = list(map(lambda x, y: str(x) + '_' + str(y), test['title'], test['event_code']))
    all_title_event_code = list(set(train["title_event_code"].unique()).union(test["title_event_code"].unique()))
    # make a list with all the unique 'titles' from the train and test set
    list_of_user_activities = list(set(train['title'].unique()).union(set(test['title'].unique())))
    # make a list with all the unique 'event_code' from the train and test set
    list_of_event_code = list(set(train['event_code'].unique()).union(set(test['event_code'].unique())))
    list_of_event_id = list(set(train['event_id'].unique()).union(set(test['event_id'].unique())))
    # make a list with all the unique worlds from the train and test set
    list_of_worlds = list(set(train['world'].unique()).union(set(test['world'].unique())))
    # create a dictionary numerating the titles
    activities_map = dict(zip(list_of_user_activities, np.arange(len(list_of_user_activities))))
    activities_labels = dict(zip(np.arange(len(list_of_user_activities)), list_of_user_activities))
    activities_world = dict(zip(list_of_worlds, np.arange(len(list_of_worlds))))
    assess_titles = list(set(train[train['type'] == 'Assessment']['title'].value_counts().index).union(set(test[test['type'] == 'Assessment']['title'].value_counts().index)))
    # replace the text titles with the number titles from the dict
    train['title'] = train['title'].map(activities_map)
    test['title'] = test['title'].map(activities_map)
    train['world'] = train['world'].map(activities_world)
    test['world'] = test['world'].map(activities_world)
    train_labels['title'] = train_labels['title'].map(activities_map)
    win_code = dict(zip(activities_map.values(), (4100*np.ones(len(activities_map))).astype('int')))

    win_code[activities_map['Bird Measurer (Assessment)']] = 4110
    
    train['timestamp'] = pd.to_datetime(train['timestamp'])
    test['timestamp'] = pd.to_datetime(test['timestamp'])
    
    # x, y
    train["x"] = train["event_data"].apply(lambda x: json.loads(x)["coordinates"]["x"] if "\"coordinates\"" in x else -999)
    train["y"] = train["event_data"].apply(lambda x: json.loads(x)["coordinates"]["y"] if "\"coordinates\"" in x else -999)
    test["x"] = test["event_data"].apply(lambda x: json.loads(x)["coordinates"]["x"] if "\"coordinates\"" in x else -999)
    test["y"] = test["event_data"].apply(lambda x: json.loads(x)["coordinates"]["y"] if "\"coordinates\"" in x else -999)
        
    return train, test, train_labels, win_code, list_of_user_activities, list_of_event_code, activities_labels, assess_titles, list_of_event_id, all_title_event_code, activities_world, list_of_worlds

# get usefull dict with maping encode
train, test, train_labels, win_code, list_of_user_activities, list_of_event_code, activities_labels, assess_titles, list_of_event_id, all_title_event_code, activities_world, list_of_worlds = encode_title(train, test, train_labels)
categoricals = ['session_title']

CPU times: user 2min 29s, sys: 3.7 s, total: 2min 32s
Wall time: 2min 32s


In [8]:
magma_short_video = [28, 2]
magma_long_video = [22]
tree_short_video = [38, 5, 24]
tree_long_video = [43, 39, 36, 40, 31, 21]
crys_short_video =  [26, 15, 25]
crys_long_video = [18, 27, 29, 19]

def get_data(user_sample, test_set=False):
    last_activity = 0
    user_activities_count = {'Clip':0, 'Activity': 0, 'Assessment': 0, 'Game':0}
    time_spent_each_act = {actv: 0 for actv in list_of_user_activities}
    event_code_count = {eve: 0 for eve in list_of_event_code}
    user_world_count = {"world_"+str(wor) : 0 for wor in activities_world.values()} #0: None, 1:CRY, 2:TREE, 3:MEGA
    clip_videos = {"magma_intro":0, "magma_long":0, "tree_intro": 0, "tree_long": 0, "crys_intro":0, "crys_long":0}
    last_session_time_sec = 0
    all_assessments = []
    accumulated_accuracy_group = 0
    accumulated_accuracy=0
    accumulated_correct_attempts = 0 
    accumulated_uncorrect_attempts = 0 
    accumulated_actions = 0
    accumulated_stumps = 0 
    accumulated_cataps = 0
    click_counts = 0
    accumulated_clicks = 0
    audio_counts = 0
    accumulated_audio_counts = 0
    animation_counts = 0
    accumulated_anime_counts = 0
    counter = 0
    cumulative_gametime = 0
    xrange = 0
    yrange = 0 
    play_game = []
    time_first_activity = float(user_sample['timestamp'].values[0])
    durations = []
    
    for i, session in user_sample.groupby('game_session', sort=False):      
        session_type = session['type'].iloc[0]
        session_title = session['title'].iloc[0]
        session_title_text = activities_labels[session_title]
        session_world = session["world"].iloc[0]
        
        # get current session time in seconds
        if session_type != 'Assessment':
            if session_type == "Clip":
                if session_title in magma_short_video:
                    clip_videos["magma_intro"] += 1
                elif session_title in magma_long_video:
                    clip_videos["magma_long"] += 1
                elif session_title in tree_short_video:
                    clip_videos["tree_intro"] += 1
                elif session_title in tree_long_video:
                    clip_videos["tree_long"] += 1
                elif session_title in crys_short_video:
                    clip_videos["crys_intro"] += 1
                elif session_title in crys_long_video:
                    clip_videos["crys_long"] += 1
                else:
                    pass
            time_spent = int(session['game_time'].iloc[-1] / 1000)
            time_spent_each_act[activities_labels[session_title]] += time_spent
            user_world_count["world_"+str(session_world)] += session.shape[0]
            
            tmp_click_counts = session["event_data"].apply(lambda x: 1 if "\"coordinates\"" in x else 0)
            click_counts += np.sum(tmp_click_counts)
            
            tmp_media = session["event_data"].apply(lambda x: json.loads(x)["media_type"] if "\"media_type\"" in x else "None").value_counts()
            if "audio" in tmp_media.index:
                audio_counts += tmp_media["audio"]
            if "animation" in tmp_media.index:
                animation_counts += tmp_media["animation"]
            
            cumulative_gametime += session.iloc[-1]["game_time"] / 1000
            
            play_game.append(session_title)
            
            if session[session.x != -999].shape[0] >= 2:
                tmp_xrange = session["x"].max() - session[session.x != -999]["x"].min()
                tmp_yrange = session["y"].max() - session[session.x != -999]["y"].min()       
            else:
                tmp_xrange = 0
                tmp_yrange = 0
            xrange = np.max([tmp_xrange, xrange])
            yrange = np.max([tmp_yrange, yrange])
            
            tmp_stump_counts = session["event_data"].apply(lambda x: np.sum(json.loads(x)["stumps"]) if "\"stumps\"" in x else 0)
            stump_counts = np.sum(tmp_stump_counts)
            accumulated_stumps += stump_counts
            #tmp_catap_counts = session["event_data"].apply(lambda x: np.sum(json.loads(x)["cataps"]) if "\"cataps\"" in x else 0)
            #catap_counts = np.sum(tmp_catap_counts)
            #accumulated_cataps += catap_counts
            
        if (session_type == 'Assessment') & (test_set or len(session)>1): # test set or session in train_label
            all_attempts = session.query(f'event_code == {win_code[session_title]}')
            true_attempts = all_attempts['event_data'].str.contains('true').sum() # 対象assessment中のtrue数
            false_attempts = all_attempts['event_data'].str.contains('false').sum() #対象assessment中のfalse数
            
            # installation_id開始から対象assessment前まで系 ------------------------
            features = user_activities_count.copy() # 対象のassessmentまでの各タイプの重複無し出現回数
            features.update(time_spent_each_act.copy()) #対象のassessmentまでの各タイトルの累積プレイ時間
            features.update(event_code_count.copy()) #対象のassessmentまでの各event_codeの累積出現回数
            features.update(user_world_count.copy()) #対象のassessmentまでの各worldの重複あり出現回数
            features.update(clip_videos.copy()) #対象のassessmentまでの各clip videoの重複あり出現回数
            features['accumulated_correct_attempts'] = accumulated_correct_attempts
            features['accumulated_uncorrect_attempts'] = accumulated_uncorrect_attempts
            accumulated_correct_attempts += true_attempts 
            accumulated_uncorrect_attempts += false_attempts
            accumulated_clicks = click_counts # 対象のassessment前までの累積click回数
            features['accumulated_clicks'] = accumulated_clicks
            accumulated_audio_counts = audio_counts # 対象のassessment前までのaudiosの累積出現回数
            features['accumulated_audios'] = accumulated_audio_counts
            accumulated_anime_counts = animation_counts # 対象のassessment前までのanimesの累積出現回数
            features['accumulated_animes'] = accumulated_anime_counts
            features['accumulated_actions'] = accumulated_actions #対象assessment前までの行数
            
            #前assessment後から次assessment前まで系 --------------------------
            # 前assess後から次assess前までの総stumps数
            features['accumulated_stumps'] = accumulated_stumps
            accumulated_stumps = 0
            #features['accumulated_cataps'] = accumulated_cataps
            #accumulated_cataps = 0
            features["total_gametime_before_assess"] = cumulative_gametime # 前assess後から次assess前までの総gameplay時間
            cumulative_gametime = 0
            features["no_playgames_before_assess"] = len(set(play_game)) # 前assess後から次assess前までのゲーム種類の数
            play_game.clear()
            features["max_xrange_before_assess"] = xrange
            features["max_yrange_before_assess"] = yrange
            xrange = 0
            yrange = 0
            features["touch_range"] = features["max_xrange_before_assess"] * features["max_yrange_before_assess"] 
            
            # 単一系 --------------------------------------------------------
            features['installation_id'] = session['installation_id'].iloc[-1]
            features['session_title'] = session_title
                                    
            # 対象assessmentの指標系 ------------------------------------------
            if durations == []: #評価対象assessmentのtimestampの期間
                features['duration_mean'] = 0
            else:
                features['duration_mean'] = np.mean(durations)
            durations.append((session.iloc[-1, 2] - session.iloc[0, 2]).seconds) 
            
            accuracy = true_attempts/(true_attempts+false_attempts) if (true_attempts+false_attempts) != 0 else 0
            if accuracy == 0:
                features['accuracy_group'] = 0
            elif accuracy == 1:
                features['accuracy_group'] = 3
            elif accuracy == 0.5:
                features['accuracy_group'] = 2
            else:
                features['accuracy_group'] = 1
            
            if test_set:
                all_assessments.append(features)
            elif true_attempts+false_attempts > 0:
                all_assessments.append(features)
                
            counter += 1
        
        n_of_event_codes = Counter(session['event_code']) #リスト中の出現回数を辞書形式で出力
        for key in n_of_event_codes.keys():
            event_code_count[key] += n_of_event_codes[key]
        
        accumulated_actions += len(session)
        if last_activity != session_type:
            user_activities_count[session_type] += 1
            last_activitiy = session_type
    if test_set:
        return all_assessments[-1]
    return all_assessments

In [9]:
new_train = []
for i, (ins_id, user_sample) in tqdm(enumerate(train.groupby('installation_id', sort=False)), total=train.installation_id.nunique(), desc='Installation_id', position=0):
    new_train += get_data(user_sample)
new_train = pd.DataFrame(new_train)
print(new_train.shape)
del train


(17690, 116)


In [10]:
new_test = []
for ins_id, user_sample in tqdm(test.groupby('installation_id', sort=False), total=test.installation_id.nunique(), desc='Installation_id', position=0):
    a = get_data(user_sample, test_set=True)
    new_test.append(a)   
new_test = pd.DataFrame(new_test)
print(new_test.shape)
del test


(1000, 116)


In [11]:
X_train = new_train.drop(['accuracy_group'],axis=1) 
lbl = preprocessing.LabelEncoder()
lbl.fit(list(X_train["installation_id"]))
X_train["installation_id"] = lbl.transform(list(X_train["installation_id"]))
remove_features = [4235, 2070]
for i in X_train.columns:
    if X_train[i].std() == 0:
        remove_features.append(i)
X_train = X_train.drop(remove_features ,axis=1)
y_train = new_train.accuracy_group

# Modelling

In [12]:
# my method
n_folds=5
groups = np.array(X_train.installation_id)
gkf=GroupKFold(n_splits = n_folds)
coefficients = []
models = []
train_qwk_scores = []
test_qwk_scores = []
lgbm_params = {
    "objective" : "regression",
    "metric" : "rmse",
    "tree_learner": "serial",
    "max_depth" : 5,
    "boosting": 'gbdt',
    "num_leaves" : 13,
    "learning_rate" : 0.01,
}
#evals_result = {}
features_list = [i for i in X_train.columns if i != "installation_id"]
feature_importance_df = pd.DataFrame(features_list, columns=["Feature"])
for i , (train_index, test_index) in enumerate(gkf.split(X_train, y_train, groups)):
    optR = OptimizedRounder()
    X_train2 = X_train.iloc[train_index,:]
    y_train2 = y_train.iloc[train_index]
    X_test2 = X_train.iloc[test_index,:]
    y_test2 = y_train.iloc[test_index]
    X_train2 = X_train2.drop(['installation_id'],axis=1) 
    X_test2 = X_test2.drop(['installation_id'],axis=1) 
    lgb_train = lgb.Dataset(X_train2, y_train2)
    lgb_eval = lgb.Dataset(X_test2, y_test2, reference=lgb_train)
    
    clf = lgb.train(
        lgbm_params, lgb_train,
        valid_sets=lgb_eval,
        num_boost_round=100000,
        early_stopping_rounds=10,
        #evals_result=evals_result,
        #feval=quadratic_weighted_kappa,
    )
    
    models.append(clf)
    train_predict = clf.predict(X_train2, num_iteration = clf.best_iteration)
    test_predict = clf.predict(X_test2, num_iteration = clf.best_iteration)
    
    optR.fit(train_predict.reshape(-1,), y_train2)
    tmp_coefficients = optR.coefficients()
    print("fold_"+str(i)+" coefficients: ", tmp_coefficients)
    opt_train_preds = optR.predict(train_predict.reshape(-1, ), tmp_coefficients)
    train_qwk_score = qwk(y_train2, opt_train_preds)
    print("training qwk: ", train_qwk_score)
    opt_test_preds = optR.predict(test_predict.reshape(-1, ), tmp_coefficients)
    test_qwk_score = qwk(y_test2, opt_test_preds)
    print("validation qwk: ", test_qwk_score)
    train_qwk_scores.append(train_qwk_score)
    test_qwk_scores.append(test_qwk_score)
    coefficients.append(tmp_coefficients)
    
    feature_importance_df["Fold_"+str(i+1)] = clf.feature_importance()
    
feature_importance_df["Average"] = np.mean(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
feature_importance_df["Std"] = np.std(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
feature_importance_df["Cv"] = feature_importance_df["Std"] / feature_importance_df["Average"]

print("                             ")
print("-----------------------------")
print('coefficients: ', coefficients)
print('train qwk list:', train_qwk_scores)
print('train qwk average score:',np.mean(train_qwk_scores))
print('valid qwk list: ', test_qwk_scores)
print('valid qwk average score:',np.mean(test_qwk_scores))

[1]	valid_0's rmse: 1.25722
Training until validation scores don't improve for 10 rounds
[2]	valid_0's rmse: 1.25406
[3]	valid_0's rmse: 1.25095
[4]	valid_0's rmse: 1.24787
[5]	valid_0's rmse: 1.24489
[6]	valid_0's rmse: 1.24192
[7]	valid_0's rmse: 1.23903
[8]	valid_0's rmse: 1.23616
[9]	valid_0's rmse: 1.23333
[10]	valid_0's rmse: 1.23053
[11]	valid_0's rmse: 1.22781
[12]	valid_0's rmse: 1.22512
[13]	valid_0's rmse: 1.22249
[14]	valid_0's rmse: 1.2199
[15]	valid_0's rmse: 1.21735
[16]	valid_0's rmse: 1.21487
[17]	valid_0's rmse: 1.21243
[18]	valid_0's rmse: 1.21004
[19]	valid_0's rmse: 1.20774
[20]	valid_0's rmse: 1.20538
[21]	valid_0's rmse: 1.20308
[22]	valid_0's rmse: 1.20081
[23]	valid_0's rmse: 1.19859
[24]	valid_0's rmse: 1.19645
[25]	valid_0's rmse: 1.19428
[26]	valid_0's rmse: 1.19217
[27]	valid_0's rmse: 1.19014
[28]	valid_0's rmse: 1.18812
[29]	valid_0's rmse: 1.18614
[30]	valid_0's rmse: 1.18424
[31]	valid_0's rmse: 1.1823
[32]	valid_0's rmse: 1.18044
[33]	valid_0's rmse: 1

In [13]:
list(X_train.columns)

['Clip',
 'Activity',
 'Assessment',
 'Game',
 'Dino Drink',
 'Egg Dropper (Activity)',
 'Chow Time',
 'Chicken Balancer (Activity)',
 'Bug Measurer (Activity)',
 'Pan Balance',
 'Scrub-A-Dub',
 'Leaf Leader',
 'Bottle Filler (Activity)',
 'Dino Dive',
 'Sandcastle Builder (Activity)',
 'Air Show',
 'Bubble Bath',
 'Fireworks (Activity)',
 'Flower Waterer (Activity)',
 'Crystals Rule',
 'All Star Sorting',
 'Watering Hole (Activity)',
 'Happy Camel',
 2050,
 4100,
 4230,
 5000,
 2060,
 4110,
 5010,
 2075,
 2080,
 2081,
 2083,
 3110,
 4010,
 3120,
 3121,
 4020,
 4021,
 4022,
 4025,
 4030,
 4031,
 3010,
 4035,
 4040,
 3020,
 3021,
 4045,
 2000,
 4050,
 2010,
 2020,
 4070,
 2025,
 2030,
 4080,
 2035,
 2040,
 4090,
 4220,
 4095,
 'world_0',
 'world_1',
 'world_2',
 'world_3',
 'magma_long',
 'tree_intro',
 'tree_long',
 'crys_long',
 'accumulated_correct_attempts',
 'accumulated_uncorrect_attempts',
 'accumulated_clicks',
 'accumulated_audios',
 'accumulated_animes',
 'accumulated_actions'

In [14]:
feature_importance_df.sort_values("Average", ascending=False).reset_index(drop=True).head(100)

,Feature,Fold_1,Fold_2,Fold_3,Fold_4,Fold_5,Average,Std,Cv
0,session_title,798,1008,996,945,981,945.6,76.776559,0.081193
1,2010,490,668,564,557,593,574.4,57.690900,0.100437
2,4070,293,498,427,499,454,434.2,75.697820,0.174339
3,accumulated_uncorrect_attempts,298,456,438,389,423,400.8,55.912074,0.139501
4,3020,293,478,338,462,419,398.0,71.501049,0.179651
5,accumulated_correct_attempts,302,264,351,361,235,302.6,48.606995,0.160631
6,Clip,194,292,281,249,270,257.2,34.649098,0.134717
7,3121,146,251,190,297,282,233.2,56.989122,0.244379
8,2030,185,207,273,211,189,213.0,31.622777,0.148464
9,3021,137,254,224,204,175,198.8,40.236302,0.202396


# prediction

In [15]:
X_test = new_test.drop(["installation_id", "accuracy_group"], axis=1)
X_test = X_test.drop(remove_features, axis=1)
pred_value = np.zeros([X_test.shape[0]])
test_coefficients = np.mean(coefficients, axis=0)
for model in models:
    pred_value += model.predict(X_test, num_iteration = model.best_iteration) /n_folds
test_pred_class= optR.predict(pred_value.reshape(-1, ), test_coefficients)
sample_submission["accuracy_group"] = test_pred_class
sample_submission.to_csv('submission.csv', index=False)

In [16]:
sample_submission["accuracy_group"].value_counts()

3    457
2    291
1    184
0     68
Name: accuracy_group, dtype: int64